In [16]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output, callback , State, ctx
import plotly.express as px
import pandas as pd

import numpy as np
import pandas as pd
import plotly
from plotly.subplots import make_subplots

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3

app = JupyterDash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options

def plot_hr(ticker, interval):
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    # ticker = input("Enter tickers")
    query1 = "select * from stock_table_hr where `ticker` == '%s' and datetime > '2023-01-01' and datetime < '2023-01-06'" % ticker
    query2 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2022-05-01' and datetime < '2023-01-06'" % ticker
    query3 = "select * from stock_table_mo where `ticker` == '%s' and datetime > '2022-01-05' and datetime < '2023-01-06'" % ticker
    Hr = pd.read_sql(query1,conn)
    Day = pd.read_sql(query2,conn)
    Mo = pd.read_sql(query3,conn)
    #declare figure
    fig = go.Figure()
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                vertical_spacing=0.1, subplot_titles=('OHLC', 'Volume'), 
                row_width=[0.2, 0.7])
    # Day['time'] = pd.to_datetime(Day['Datetime'])
    # Day.set_index(Day['time'],inplace = True)

    #Candlestick
    for column in [Hr,Day,Mo]:
        # name = column.Ticker[0]
        column['time'] = pd.to_datetime(column['Datetime'])
        column['MA50'] = column['Close'].rolling(window = 50 , min_periods = 0).mean()
        column['MA200'] = column['Close'].rolling(window = 200 , min_periods = 0).mean()
        column.set_index(column['time'],inplace = True)
        fig.add_trace(go.Candlestick(x=column.Datetime,
                        open=column.Open,
                        high=column.High,
                        low=column.Low,
                        close=column.Close, 
                        name = ticker,),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA50',x=column.Datetime,y=column['MA50']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA200',x=column.Datetime,y=column['MA200']),row=1, col=1)
        fig.add_trace(go.Bar(x=column.Datetime, y=column.Volume,showlegend=False), row=2, col=1)
        
    df_resample_hr = Hr.resample('H').max()
    merged_index_hr  = Hr.index.append(df_resample_hr.index)
    timegap_hr = merged_index_hr[~merged_index_hr.duplicated(keep = False)]

    df_resample_day = Day.resample('D').max()
    merged_index_day  = Day.index.append(df_resample_day.index)
    timegap_day = merged_index_day[~merged_index_day.duplicated(keep = False)]

    # fig.update_xaxes( rangebreaks=[ dict(values = timegap_hr , dvalue = 3600000)])
    if interval == "Hour":
        fig.update_xaxes(rangebreaks=[dict(values=timegap_hr, dvalue=3600000)])
        
    elif interval == "Day":
        fig.update_xaxes(rangebreaks=[dict(values=timegap_day)])
    # fig.update_xaxes( rangebreaks=[ dict(values = timegap_hr , dvalue = 3600000)])


    fig.update_yaxes(rangemode="nonnegative")
    fig.update_xaxes(rangemode="nonnegative")
    fig.update_yaxes(fixedrange=False)
    fig.update_layout(hovermode = "x")
    fig.update(layout_xaxis_rangeslider_visible=False)
    return fig

def plot_d(ticker):
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    # ticker = input("Enter tickers")
    query2 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2022-05-01' and datetime < '2023-01-06'" % ticker
    Day = pd.read_sql(query2,conn)
    #declare figure
    fig = go.Figure()
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                vertical_spacing=0.1, subplot_titles=('OHLC', 'Volume'), 
                row_width=[0.2, 0.7])
    # Day['time'] = pd.to_datetime(Day['Datetime'])
    # Day.set_index(Day['time'],inplace = True)

    #Candlestick
    Day['time'] = pd.to_datetime(Day['Datetime'])
    Day['MA50'] = Day['Close'].rolling(window = 50 , min_periods = 0).mean()
    Day['MA200'] = Day['Close'].rolling(window = 200 , min_periods = 0).mean()
    Day.set_index(Day['time'],inplace = True)
    fig.add_trace(go.Candlestick(x=Day.Datetime,
                    open=Day.Open,
                    high=Day.High,
                    low=Day.Low,
                    close=Day.Close, 
                    name = ticker,),row=1, col=1)
    fig.add_trace(go.Scatter(name = 'MA50',x=Day.Datetime,y=Day['MA50']),row=1, col=1)
    fig.add_trace(go.Scatter(name = 'MA200',x=Day.Datetime,y=Day['MA200']),row=1, col=1)
    fig.add_trace(go.Bar(x=Day.Datetime, y=Day.Volume,showlegend=False), row=2, col=1)

    df_resample_day = Day.resample('D').max()
    merged_index_day  = Day.index.append(df_resample_day.index)
    timegap_day = merged_index_day[~merged_index_day.duplicated(keep = False)]

    # fig.update_xaxes( rangebreaks=[ dict(values = timegap_hr , dvalue = 3600000)])

    fig.update_xaxes(rangebreaks=[dict(values=timegap_day)])

    fig.update_yaxes(rangemode="nonnegative")
    fig.update_xaxes(rangemode="nonnegative")
    fig.update_yaxes(fixedrange=False)
    fig.update_layout(hovermode = "x")
    fig.update(layout_xaxis_rangeslider_visible=False)
    return fig

def plot_mo(ticker):
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    # ticker = input("Enter tickers")
    query3 = "select * from stock_table_mo where `ticker` == '%s' and datetime > '2022-01-05' and datetime < '2023-01-06'" % ticker
    Mo = pd.read_sql(query3,conn)
    #declare figure
    fig = go.Figure()
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                vertical_spacing=0.1, subplot_titles=('OHLC', 'Volume'), 
                row_width=[0.2, 0.7])

    #Candlestick
    Mo['time'] = pd.to_datetime(Mo['Datetime'])
    Mo['MA50'] = Mo['Close'].rolling(window = 50 , min_periods = 0).mean()
    Mo['MA200'] = Mo['Close'].rolling(window = 200 , min_periods = 0).mean()
    Mo.set_index(Mo['time'],inplace = True)
    fig.add_trace(go.Candlestick(x=Mo.Datetime,
                    open=Mo.Open,
                    high=Mo.High,
                    low=Mo.Low,
                    close=Mo.Close, 
                    name = ticker,),row=1, col=1)
    fig.add_trace(go.Scatter(name = 'MA50',x=Mo.Datetime,y=Mo['MA50']),row=1, col=1)
    fig.add_trace(go.Scatter(name = 'MA200',x=Mo.Datetime,y=Mo['MA200']),row=1, col=1)
    fig.add_trace(go.Bar(x=Mo.Datetime, y=Mo.Volume,showlegend=True), row=2, col=1)


    fig.update_yaxes(rangemode="nonnegative")
    fig.update_xaxes(rangemode="nonnegative")
    fig.update_yaxes(fixedrange=False)
    fig.update_layout(hovermode = "x")
    fig.update(layout_xaxis_rangeslider_visible=False)
    return fig

def stock_name():
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    query = "select distinct Ticker from stock_table_hr"
    r_df = pd.read_sql(query,conn)
    list_db = r_df['Ticker'].values.tolist()
    return list_db


#DASH
app.layout = html.Div(children=[
    html.H1(children='Stock Radar'),
    html.Div([
        "Stock : ",
        dcc.Input(id='ticker-input', value='', type='text'),
        html.Button(id='submit-button', n_clicks=0, children='Submit'),
        dcc.Dropdown(['Hour', 'Day', 'Month'], searchable=False, id='stock-dropdown')
    ]),
    html.Div(id='output-div'),

    dcc.Graph(
        id='graph',
        figure=plot_hr('','Hour')
    ),
    dcc.ConfirmDialog(
        id='popup',
        message='Not found that tickers',
        displayed=False,
    ),
    dcc.ConfirmDialog(
        id='popup_dl',
        message='Downloaded',
        displayed=False,
    ),
])

@app.callback([Output('graph','style'),Output('popup', 'displayed')],
              [Input('submit-button', 'n_clicks')],
              [State('ticker-input', 'value')])

def update_popup_and_graph(n_clicks, value):
    list_db = stock_name()
    value = value.upper()+ ".BK"
    if n_clicks:
        if value.strip() == '' or (value.strip() not in list_db):
            return plot_hr('','Hour'),True
        else:
            return plot_hr(value, 'Hour'),False
    return plot_hr('', 'Hour'),False


@app.callback(Output('graph','figure'),
              [Input('stock-dropdown', 'value'), 
              Input('submit-button', 'n_clicks')],
              [State('ticker-input', 'value')])
def update_graph(stock_dropdown, submit_clicks, ticker_input):
    print(stock_dropdown)
    if stock_dropdown != None:
        return dropdown_output(stock_dropdown, "ACE.BK")
    elif submit_clicks is not None:
        return update_output(ticker_input)
    

def update_output(ticker_input):
    list_db = stock_name()
    value = ticker_input.upper()+ ".BK"
    if value.strip() == '' or (value.strip() not in list_db):
            fig = plot_hr('','Hour')
            return fig
    else:
        value = value.upper()
        fig = plot_hr(value,'Hour')
        return fig

def dropdown_output(stock_dropdown ,ticker_input):
    fig = plot_hr(ticker_input, stock_dropdown)
    return fig

@app.callback(
    Output("popup_dl", "displayed"),
    [Input("popup", "submit_n_clicks")],
    [State('ticker-input', 'value')]
)
def execute_function(submit_n_clicks ,value):
    if submit_n_clicks:
        return True
    else:
        pass
    return False

@app.callback(Output('stock-dropdown', 'value'),
              [Input('submit-button', 'n_clicks')]
             )
def reset_dropdown(n_clicks):
    return None

@app.callback(
    Output("ticker-input", "value"),
    [Input("submit-button", "n_clicks")],
    )
def clear_input(n_clicks):
    return ""



app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


None
Hour
None
Hour


In [11]:
from jupyter_dash import JupyterDash
from dash import Dash, Input, Output, ctx, html, dcc
import plotly.express as px
import plotly.graph_objects as go

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Button('Draw Graph', id='draw'),
    html.Button('Reset Graph', id='reset'),
    dcc.Graph(id='graph')
])

@app.callback(
    Output('graph', 'figure'),
    Input('reset', 'n_clicks'),
    Input('draw', 'n_clicks'),
    prevent_initial_call=True
)
def update_graph(b1, b2):
    triggered_id = ctx.triggered_id
    print(triggered_id)
    if triggered_id == 'reset':
         return reset_graph()
    elif triggered_id == 'draw':
         return draw_graph()

def draw_graph():
    df = px.data.iris()
    return px.scatter(df, x=df.columns[0], y=df.columns[1])

def reset_graph():
    return go.Figure()

app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


draw
reset
